In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('0_parameters.sh')

cat_or_zcat <- function(f){
    ifelse(endsWith(f, '.zst'), 'zstdcat', ifelse(endsWith(f, '.gz'), 'zcat', 'cat'))
}


In [3]:
ukb_keep_wb %>% fread(head=F, colClasses = 'character') %>% rename('FID'=1, 'IID'=2) -> keep_df


In [4]:
file.path(data_scratch_d, 'UKB_PRS_PheWAS/HGIrel5.PRS_PheWAS.1e-5.tsv.gz') %>%
fread(select=c('GBE_ID')) %>% pull() %>% unique() -> phe_list


In [5]:
phe_f <- ukb_master_phe

fread(
    cmd=paste(cat_or_zcat(phe_f), phe_f),
    colClasses = c('#FID'='character', 'IID'='character'),
    select = all_of(c('#FID', 'IID', 'age', 'sex', 'Array', paste0('PC', 1:10), phe_list))
) %>% rename('FID'='#FID') -> phe_df


In [8]:
phe_df %>%
inner_join(keep_df, by=c('FID', 'IID')) %>%
mutate(age_sq = age * age) %>%
select(all_of(c('FID', 'IID', 'age', 'age_sq', 'sex', 'Array', paste0('PC', 1:10), phe_list))) -> full_df


In [9]:
min(full_df$age)

[1] 47

In [10]:
max(full_df$age)

[1] 81

In [11]:
full_df %>%
rename('#FID' = 'FID') %>%
fwrite(file.path(data_d, 'UKB_PRS_PheWAS_follow_up_GWAS', 'phe.tsv'), sep='\t', na = "NA", quote=F)
